# ESGF Virtual Aggregation - Projected climate change signals and uncertainty under global warming levels

> This notebook is a reproducibility example of the IPCC-WGI AR6 Interactive Atlas products, which has been adapted to work with the ESGF Virtual Aggregation. This work is licensed under a [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0).
>
> ![Creative Commons License >](https://i.creativecommons.org/l/by/4.0/88x31.png)

**E. Cimadevilla** (Santander Meteorology Group. Instituto de Física de Cantabria, CSIC-UC, Santander, Spain).

This notebook is an **example of the calculation and visualization of the IPCC-WGI AR6 uncertainty methods (simple and advanced) for projected delta changes**. Please refer to the **AR6 WGI Cross-Chapter Box Atlas 1** ([Gutiérrez et al., 2021](https://www.ipcc.ch/report/ar6/wg1/chapter/atlas/)) for more information. We also **introduce the Global Warming Level dimension for the analysis of the climate change signal**.

This notebook works with data available in the ESGF, which is accessed in a **remote data access** fashion through virtual aggregations provided by the [ESGF Virtual Aggregation](https://doi.org/10.5194/gmd-2024-120).

In [ ]:
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cftime

import xarray
import dask

In [ ]:
dask.config.set(scheduler="processes")

## 1. The Global Warming Level analysis dimension

Instead of calculating climate change anomalies for a fixed period, we will calculate the anomaly for a given level of global warming (GWL). To do so, we need the information on the **time windows where the global surface temperature reaches the different levels of warming**. This information is available at the [IPCC-WGI/Atlas GitHub repository](https://github.com/IPCC-WG1/Atlas/tree/main/notebooks).

Note that we have pointed to the `CMIP6_Atlas_WarmingLevels.csv` file, as we are going to work with CMIP6 data. In this example, we will focus on the +3ºC GWL. We will consider the ssp585 scenario, however, any other scenario can be considered, as the anomalies do not vary significantly across scenarios when the GWL dimension is analyzed.

In [ ]:
url = "https://github.com/SantanderMetGroup/ATLAS/raw/refs/heads/main/warming-levels/CMIP6_Atlas_WarmingLevels.csv"
gwls_file = url.split("/")[-1]
with open(gwls_file, "w") as f:
    response = requests.get(url)
    response.raise_for_status()
    f.write(response.text)
gwls = pd.read_csv(gwls_file)
model_runs = gwls["model_run"].str.split("_", expand=True).set_index(0)[1].to_dict()

## 2. Data loading for the different GWLs

To locate the data in which we are interested, we use the CSV inventory of the ESGF Virtual Aggregation. We will analyze the inventory using standard Pandas functionality. The data will be loaded in a remote data access fashion via OPeNDAP provided by a THREDDS Data Server.

The versions of the ESGF datasets in which we are interested are available in the `atlas-pr.csv' file provided in this same directory. These versions were extracted from the [IPCC-WGI/Atlas GitHub repository](https://raw.githack.com/IPCC-WG1/Atlas/devel/data-sources/CMIP6/CMIP6.html).

In [ ]:
esgf_va_catalog = pd.read_csv("esgf-va_catalog.csv.zip").fillna("")
atlas_ipcc_catalog = pd.read_csv("atlas-pr.csv")

The following function manages the locations of the datasets from the ESGF Virtual Aggregation that we are interested in. Since some versions are no longer available on the ESGF—having been replaced by newer versions—not all of the original datasets will be loaded.

In [ ]:
def find_model_runs(esgf_va_catalog, atlas_ipcc_catalog, model_runs, experiment):
    ignored_models = [
        "EC-Earth3-Veg", # replica unavailable
        "BCC-CSM2-MR", # time units change across netCDF files
        "KACE-1-0-G", # there is no ssp585 for this model
        "UKESM1-0-LL", # there is no ssp585 for this model
        "MIROC-ES2L", # no replica seems to work

        # not available later for Pangeo
        "ACCESS-CM2",
        "ACCESS-ESM1-5",

        # temporal removes
        "KIOST-ESM",
    ]
    
    preferred_replicas = {
        "historical": {
            "EC-Earth3-Veg": "esgf3.dkrz.de",
            "HadGEM3-GC31-LL": "", # esgf-data04.diasjp.net netcdf-java error, ...
        },
        "ssp585": {
            "EC-Earth3-Veg": "esgf3.dkrz.de",
            "HadGEM3-GC31-LL": "",
        },
    }
    
    esgf_va_model_runs = []
    for model in model_runs:
        if model in ignored_models:
            continue
    
        esgf_va_subset = esgf_va_catalog.query(f"project == 'CMIP6' & model == '{model}' & table == 'day' & variable == 'pr' & experiment == '{experiment}'").set_index(["project", "model", "experiment", "version"]).drop(["variable"], axis=1)
        atlas_ipcc_subset = atlas_ipcc_catalog.drop(["variable"], axis=1)
        
        subset = esgf_va_subset.join(atlas_ipcc_subset.set_index(["project", "model", "experiment", "version"]), on=["project", "model", "experiment", "version"], how="inner")
        if len(subset) > 0:
            # Choose the preferred replica or default to esgf.ceda.ac.uk
            if model in preferred_replicas[experiment]:
                replica = preferred_replicas[experiment][model]
            else:
                replica = "esgf.ceda.ac.uk"
    
            # Probe if the required model member is included in the esgf-va
            url = subset[subset["replica"] == replica]["location"].iloc[0]
            print(url)
            ds = xarray.open_dataset(
                url,
                decode_times=False,
                drop_variables=["tracking_id", "further_info_url", "time_bnds", "lat_bnds", "lon_bnds"])
    
            if model_runs[model].encode("ascii") in ds["variant_label"].values:
                record = {"url": url, "model_run": model + "_" + model_runs[model]}
                esgf_va_model_runs.append(record)

    return pd.DataFrame.from_records(esgf_va_model_runs)

Locate the datasets for both the historical and future scenario periods.

In [ ]:
ssp585_model_runs = find_model_runs(esgf_va_catalog, atlas_ipcc_catalog, model_runs, "ssp585")
historical_model_runs = find_model_runs(esgf_va_catalog, atlas_ipcc_catalog, model_runs, "historical")

We now create a DataFrame containing all the available datasets.

In [ ]:
df = historical_model_runs.set_index("model_run").join(
    ssp585_model_runs.set_index("model_run"), how="inner", lsuffix="h", rsuffix="s").rename(
    {"urlh": "historical", "urls": "ssp585"}, axis=1)
df

Store the inventory of URLs for future usages.

In [ ]:
# df.to_csv("esgf-va_pr_ceda.csv")

Open the datasets using [xarray](https://docs.xarray.dev/en/stable/).

In [ ]:
def load_datasets(locations, members, experiment):
    dss = []
    for location in locations:
        name = location.split("/")[-1]
        model = name.split("_")[3]

        ds = xarray.open_dataset(
            location,
            drop_variables=["tracking_id", "further_info_url", "time_bnds", "lat_bnds", "lon_bnds"]).sel(
            variant_label=members[model].encode("ascii"))

        if experiment == "ssp585":
            ds = ds.isel(time=slice(None, 31390))
            # limit to 2100, avoid cf time indexing issues
            if isinstance(ds["time"][0].item(), cftime.Datetime360Day):
                ds = ds.sel(time=slice("20150101", "21001230"))
            else:
                ds = ds.sel(time=slice("20150101", "21001231"))
        elif experiment == "historical":
            pass
        else:
            raise ValueError("Invalid experiment... exiting")
    
        # add the model_run global attribute
        ds.attrs["model_run"] = f"{model}_{members[model]}"
        dss.append(ds)

    return dss

In [ ]:
historical_dss = load_datasets(df["historical"], model_runs, "historical")
ssp585_dss = load_datasets(df["ssp585"], model_runs, "ssp585")

Print size of the datasets:

In [ ]:
for ds in historical_dss:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

In [ ]:
for ds in ssp585_dss:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

## 3. Data retrieval and subsetting

From the previous section, we observe that the datasets are relatively large, which would require substantial data transfer from the ESGF data nodes to the client executing the climate data analysis workflow. However, since we are focusing only on the European region, we can take advantage of remote data access subsetting capabilities to retrieve only the necessary data from the ESGF nodes to support the workflow.

To achieve this, we will implement a function responsible for efficiently fetching the required data.

In [ ]:
def efficient_retrieval_and_compute(ds):
    lats, lons = slice(35, 72), slice(-11, 35) # Europe in (-180, 180)

    # handle Europe being split when using (0, 360)
    if ds["lon"].min().item() >= 0:
        left = ds.sel(lon=slice(0, 36), lat=lats).chunk({"time": 1000})
        right = ds.sel(lon=slice(348, 360), lat=lats).chunk({"time": 1000})

        spatial = xarray.concat([left, right], dim="lon")
    else:
        spatial = ds.sel(lon=lons, lat=lats)

    winter = spatial.sel(time=spatial.time.dt.month.isin([12,1,2])).resample({"time": "MS"}).sum().load()
    winter = winter.assign_coords(lon=((winter["lon"] + 180) % 360) - 180).sortby("lon")

    return winter

In [ ]:
%%time

historical_dss_subset = [efficient_retrieval_and_compute(ds) for ds in historical_dss]

In [ ]:
%%time

ssp585_dss_subset = [efficient_retrieval_and_compute(ds) for ds in ssp585_dss]

Now that we have loaded all the required data via subsetting and remote data access, we can examine the actual amount of data needed to carry out this workflow. We can observe that the data required to carry out the workflow is much smaller compared to the full size of the datasets.

In [ ]:
for ds in historical_dss_subset:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

In [ ]:
for ds in ssp585_dss_subset:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

## 4. Interpolation to a common grid

To perform the model agreement task, we need to interpolate to the same grid of the original Interactive Atlas dataset. We will obtain the grid from the original dataset.

In [ ]:
iad_catalog = pd.read_csv("https://raw.githubusercontent.com/SantanderMetGroup/IPCC-Atlas-Datalab/refs/heads/main/data_inventory.csv")
iad_subset = iad_catalog.query('type == "opendap" & variable == "pr" & project == "CMIP6" & experiment == "ssp585" & frequency == "mon"')
iad_location = iad_subset["location"].iloc[0]
iad_location

In [ ]:
iad = xarray.open_dataset(iad_location).sel(lat=slice(35, 72), lon=slice(-11, 35))
iad["member"] = [b"_".join(member.split(b"_")[1:]).decode("ascii") for member in iad["member"].values]
iad["pr"]

Once we have got access to the Interactive Atlas dataset, we perform interpolation.

In [ ]:
ssp585_dss_subset_interp = [ds.interp(
    lat=iad["lat"],
    lon=iad["lon"],
    method="linear",
    kwargs={"fill_value": "extrapolate"})
              for ds in ssp585_dss_subset]

historical_dss_subset_interp = [ds.interp(
    lat=iad["lat"],
    lon=iad["lon"],
    method="linear",
    kwargs={"fill_value": "extrapolate"})
              for ds in historical_dss_subset]

In [ ]:
for ds in ssp585_dss_subset_interp:
    ds["time"] = ds["time"].astype("datetime64[ns]")
    ds["pr"] = xarray.where(ds["pr"] < 0, 0, ds["pr"])

for ds in historical_dss_subset_interp:
    ds["time"] = ds["time"].astype("datetime64[ns]")
    ds["pr"] = xarray.where(ds["pr"] < 0, 0, ds["pr"])

In [ ]:
cmip6_ssp585 = xarray.concat(
    ssp585_dss_subset_interp,
    xarray.Variable("member", [ds.attrs["model_run"] for ds in ssp585_dss_subset_interp]))

cmip6_historical = xarray.concat(
    historical_dss_subset_interp,
    xarray.Variable("member", [ds.attrs["model_run"] for ds in historical_dss_subset_interp]))

## 5. Global warming levels

Since we are working with warming levels and need to load a different period of years for each model, we will first extract the information about the members from the inventory. This information is essential to match the GWL array obtained earlier. In other words, we need to create an index object that provides the model position in the data for each row in the gwl3 object. 

In [ ]:
gwls3 = gwls[["model_run", "3_ssp585"]]
model_run_gwl = df.join(gwls3.set_index("model_run"), on="model_run")["3_ssp585"]
model_run_gwl

Extract a 20-year period for each model: 10 years before and 10 years after the model reaches the GWL.

In [ ]:
cmip6_ssp585_gwl = xarray.concat(
    [cmip6_ssp585.sel(
        member=model_run,
        time=slice(f"{year-10}1201", f"{year+10}0201"))
     for model_run, year in model_run_gwl.reset_index().values
     if year != 9999],
    "member") * 2880

We perform an annual aggregation of the variable. Additionally, we generate a time-series plot to illustrate the concept of GWLs, where each model spans a different time period.

In [ ]:
qmean = cmip6_ssp585_gwl.mean(["lat", "lon"]).resample(time="QS-DEC").mean()
plot = qmean["pr"].sel(time=qmean["pr"].time.dt.month==12).plot.line(x="time", add_legend=False, figsize=(15, 5))

Now we plot the map of the climatologies (mean of the period; 20-year means in this case). In the resulting plot, each panel represents a CMIP6 member (a model).

In [ ]:
lats, lons = slice(35, 72), slice(-11, 35)

plot = cmip6_ssp585_gwl["pr"].mean("time").plot(
    x="lon", y="lat", col="member", col_wrap=6,
    # vmin=0, vmax=13,
    figsize=(28,13),
    add_colorbar=True,
    cmap="coolwarm_r",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

To get the climate change signal, we first need to load data from the historical scenario. To do this we repeat the filtering process to get the required dataset.

We are considering the pre-industrial period. In this case, the reference period is common to all models.

In [ ]:
cmip6_historical_gwl = xarray.concat(
    [cmip6_historical.sel(
        member=model_run,
        time=slice("18501201", "19000201"))
     for model_run, year in model_run_gwl.reset_index().values
     if year != 9999],
    "member") * 2880

Now we can calculate the anomaly by computing the difference between both climatologies. We can easily do this using xarray functions.

In [ ]:
delta = cmip6_ssp585_gwl["pr"].mean("time") - cmip6_historical_gwl["pr"].mean("time")

In [ ]:
plot = delta.plot(
    x="lon", y="lat", col="member", col_wrap=6,
    figsize=(28,13),
    add_colorbar=True,
    cmap="coolwarm_r",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

These maps represent the climate change signal as an absolute anomaly. We can calculate the relative anomaly as follows.

In [ ]:
delta_rel = (delta / cmip6_historical_gwl["pr"].mean("time")) * 100

In [ ]:
plot = delta_rel.plot(
    x="lon", y="lat", col="member", col_wrap=6,
    figsize=(28,13),
    vmax=200, vmin=-200,
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

We now calculate and plot the multi-model anomaly mean.

In [ ]:
ens_mean = delta_rel.mean("member")

plot = ens_mean.plot(
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    vmin=-50, vmax=50,
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

plot.axes.coastlines()

## 6. Uncertainty calculation and representation

We implement both the "simple" and "advanced" methods for uncertainty characterization defined in the IPCC Sixth Assessment Report. Please refer to the AR6 WGI Cross-Chapter Box Atlas 1 (Gutiérrez et al., 2021) for more information.

In [ ]:
def model_agreement(da, axis, th = 80):
    nmembers, nlat, nlon = da.shape
    mask = np.array([
        (da[:,i,j] > 0).sum() > int(nmembers * th / 100)
        if da[:,i,j].mean() > 0
        else (da[:,i,j] < 0).sum() > int(nmembers * th / 100)
        for i in range(nlat)
        for j in range(nlon)]).reshape((nlat,nlon))
    
    return mask

def hatching(plot, mask, data):
    rows, cols = mask.shape
    for i in range(rows):
        for j in range(cols):
            lat, lon = data["lat"][i].item(), data["lon"][j].item()
            if mask[i,j]:
                plot.axes.plot([lon-.5,lon+.5],[lat+.5,lat-.5],'-',c="black", linewidth=.5)


Compute and display uncertainty via hatching using the "simple" method.

In [ ]:
mask_simple = delta.reduce(model_agreement, "member")

plot = ens_mean.plot(
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    vmin=-50, vmax=50,
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

hatching(plot, ~mask_simple, ens_mean)

plot.axes.set_title(f"ESGF-VA ({len(delta['member'])} models)")
plot.axes.coastlines()

plt.savefig("uncer-europe-esgfva.pdf", bbox_inches="tight")

We now display regional precipitation stripes for Europe. Each model represents a 20-year period centered on the point at which it reaches the GWL.

In [ ]:
cmip6_ssp585_y = cmip6_ssp585_gwl["pr"].resample(time="QS-DEC").mean()
cmip6_ssp585_y = cmip6_ssp585_y.sel(time=cmip6_ssp585_y["time"].dt.month==12)

cmip6_hist_c = cmip6_historical_gwl["pr"].mean(["time"])

year_delta = cmip6_ssp585_y - cmip6_hist_c
year_delta_rel = year_delta / cmip6_hist_c * 100

regional_mean = year_delta_rel.mean(["lat", "lon"])
regional_mean["member"] = [x.split("_")[0] for x in regional_mean["member"].values]

regional_mean.plot.imshow(
    figsize=(14,3.5),
    add_colorbar=True,
    cmap="BrBG",
    vmin=-50,vmax=50)
plt.savefig("regional-stripes-pr.pdf", bbox_inches="tight")

## 7. Pangeo

Until now, we have been working with the datasets provided by the ESGF Virtual Aggregation. We now carry out the same climate data analysis workflow by using the Analysis-Ready Cloud-Optimized datasets provided by [Pangeo](https://doi.org/10.1109/MCSE.2021.3059437).

In [ ]:
pangeo_catalog = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
pangeo_catalog["version"] = pangeo_catalog["version"].astype(str)
pangeo_catalog.head()

Find the same model runs used within the ESGF Virtual Aggregation.

In [ ]:
def find_model_runs_pangeo(pangeo_catalog, atlas_ipcc_catalog, model_runs, esgf_va_df, experiment):
    pangeo_model_runs = []
    
    for model, run in esgf_va_df.index.str.split("_"):
        version = atlas_ipcc_catalog.query(f"model == '{model}' & experiment == '{experiment}' & member == '{run}' & variable == 'pr'")["version"].iloc[0][1:]
        subset = pangeo_catalog.query(f"source_id == '{model}' & table_id == 'day' & variable_id == 'pr' & experiment_id == '{experiment}' & member_id == '{run}' & version == '{version}'").set_index(["source_id", "experiment_id", "version"]).drop(["variable_id"], axis=1)

        if model in ["ACCESS-CM2","ACCESS-ESM1-5"]:
            continue

        url = subset["zstore"].iloc[0]
        record = {"url": url, "model_run": model + "_" + model_runs[model]}
        pangeo_model_runs.append(record)

    return pd.DataFrame.from_records(pangeo_model_runs)

In [ ]:
ssp585_model_runs_pangeo = find_model_runs_pangeo(pangeo_catalog, atlas_ipcc_catalog, model_runs, df, "ssp585")
historical_model_runs_pangeo = find_model_runs_pangeo(pangeo_catalog, atlas_ipcc_catalog, model_runs, df, "historical")

In [ ]:
df_pangeo = historical_model_runs_pangeo.set_index("model_run").join(ssp585_model_runs_pangeo.set_index("model_run"), how="inner", lsuffix="h", rsuffix="s").rename({"urlh": "historical", "urls": "ssp585"}, axis=1)
df_pangeo

Load the datasets.

In [ ]:
def load_datasets_pangeo(locations, members, experiment):
    dss = []
    for location in locations:
        model = location.split("/")[6]

        ds = xarray.open_zarr(
            location,
            drop_variables=["time_bnds", "lat_bnds", "lon_bnds"])

        if experiment == "ssp585":
            ds = ds.isel(time=slice(None, 31390))
            # limit to 2100, avoid cf time indexing issues
            if isinstance(ds["time"][0].item(), cftime.Datetime360Day):
                ds = ds.sel(time=slice("20150101", "21001230"))
            else:
                ds = ds.sel(time=slice("20150101", "21001231"))
        elif experiment == "historical":
            pass
        else:
            raise ValueError("Invalid experiment... exiting")
    
        # add the model_run global attribute
        ds.attrs["model_run"] = f"{model}_{members[model]}"
        dss.append(ds)

    return dss

In [ ]:
historical_dss_pangeo = load_datasets_pangeo(df_pangeo["historical"], model_runs, "historical")
ssp585_dss_pangeo = load_datasets_pangeo(df_pangeo["ssp585"], model_runs, "ssp585")

Print size of the datasets:

In [ ]:
for ds in historical_dss_pangeo:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

In [ ]:
for ds in ssp585_dss_pangeo:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

Implement the function that will subset and fetch the data from the cloud repository.

In [ ]:
def efficient_retrieval_and_compute_pangeo(ds):
    lats, lons = slice(35, 72), slice(-11, 35) # Europe in (-180, 180)

    # handle Europe being split when using (0, 360)
    if ds["lon"].min().item() >= 0:
        spatial = ds.sel(
            time=ds.time.dt.month.isin([12,1,2]),
            lon=(ds["lon"] <= 36) | (ds["lon"] >= 348),
            lat=slice(35, 72)).load()
    else:
        spatial = ds.sel(time=ds.time.dt.month.isin([12,1,2]), lon=lons, lat=lats).load()

    winter = spatial.resample({"time": "MS"}).sum()
    winter = winter.assign_coords(lon=((winter["lon"] + 180) % 360) - 180).sortby("lon")

    return winter

Load the data.

In [ ]:
%%time

historical_dss_subset_pangeo = [efficient_retrieval_and_compute_pangeo(ds) for ds in historical_dss_pangeo]

In [ ]:
%%time

ssp585_dss_subset_pangeo = [efficient_retrieval_and_compute_pangeo(ds) for ds in ssp585_dss_pangeo]

Look at the sizes of the subsets.

In [ ]:
for ds in historical_dss_subset_pangeo:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

In [ ]:
for ds in ssp585_dss_subset_pangeo:
    print(f'{ds["pr"].size * ds["pr"].dtype.itemsize / 2**20:.2f} MB')

Interpolate to a common grid.

In [ ]:
ssp585_dss_subset_interp_pangeo = [ds.interp(
    lat=iad["lat"],
    lon=iad["lon"],
    method="linear",
    kwargs={"fill_value": "extrapolate"})
              for ds in ssp585_dss_subset_pangeo]

historical_dss_subset_interp_pangeo = [ds.interp(
    lat=iad["lat"],
    lon=iad["lon"],
    method="linear",
    kwargs={"fill_value": "extrapolate"})
              for ds in historical_dss_subset_pangeo]

for ds in ssp585_dss_subset_interp_pangeo:
    ds["time"] = ds["time"].astype("datetime64[ns]")
    ds["pr"] = xarray.where(ds["pr"] < 0, 0, ds["pr"])

for ds in historical_dss_subset_interp_pangeo:
    ds["time"] = ds["time"].astype("datetime64[ns]")
    ds["pr"] = xarray.where(ds["pr"] < 0, 0, ds["pr"])

In [ ]:
cmip6_ssp585_pangeo = xarray.concat(
    ssp585_dss_subset_interp_pangeo,
    xarray.Variable("member", [ds.attrs["model_run"] for ds in ssp585_dss_subset_interp_pangeo]))

cmip6_historical_pangeo = xarray.concat(
    historical_dss_subset_interp_pangeo,
    xarray.Variable("member", [ds.attrs["model_run"] for ds in historical_dss_subset_interp_pangeo]))

Reproduce the figures.

In [ ]:
cmip6_ssp585_gwl = xarray.concat(
    [cmip6_ssp585_pangeo.sel(
        member=model_run,
        time=slice(f"{year-10}1201", f"{year+10}0201"))
     for model_run, year in model_run_gwl.reset_index().values
     if year != 9999],
    "member") * 2880

In [ ]:
qmean = cmip6_ssp585_gwl.mean(["lat", "lon"]).resample(time="QS-DEC").mean()
plot = qmean["pr"].sel(time=qmean["pr"].time.dt.month==12).plot.line(x="time", add_legend=False, figsize=(15, 5))

In [ ]:
lats, lons = slice(35, 72), slice(-11, 35)

plot = cmip6_ssp585_gwl["pr"].mean("time").plot(
    x="lon", y="lat", col="member", col_wrap=6,
    # vmin=0, vmax=13,
    figsize=(28,13),
    add_colorbar=True,
    cmap="coolwarm_r",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

In [ ]:
cmip6_historical_gwl = xarray.concat(
    [cmip6_historical_pangeo.sel(
        member=model_run,
        time=slice("18501201", "19000201"))
     for model_run, year in model_run_gwl.reset_index().values
     if year != 9999],
    "member") * 2880

In [ ]:
delta = cmip6_ssp585_gwl["pr"].mean("time") - cmip6_historical_gwl["pr"].mean("time")

In [ ]:
plot = delta.plot(
    x="lon", y="lat", col="member", col_wrap=6,
    figsize=(28,13),
    add_colorbar=True,
    cmap="coolwarm_r",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

In [ ]:
delta_rel = (delta / cmip6_historical_gwl["pr"].mean("time")) * 100

In [ ]:
plot = delta_rel.plot(
    x="lon", y="lat", col="member", col_wrap=6,
    figsize=(28,13),
    vmax=200, vmin=-200,
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

for ax in plot.axs.flatten():
    ax.coastlines()
    ax.set_extent((lons.start, lons.stop, lats.start, lats.stop), ccrs.PlateCarree())

Ensemble mean.

In [ ]:
ens_mean = delta_rel.mean("member")

plot = ens_mean.plot(
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    vmin=-50, vmax=50,
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

plot.axes.coastlines()

Model agreement.

In [ ]:
mask_simple = delta.reduce(model_agreement, "member")

plot = ens_mean.plot(
    add_colorbar=True,
    cmap="BrBG",
    cbar_kwargs={"shrink": .5},
    vmin=-50, vmax=50,
    subplot_kws=dict(projection=ccrs.PlateCarree(central_longitude=0)),
    transform=ccrs.PlateCarree())

hatching(plot, ~mask_simple, ens_mean)

plot.axes.set_title(f"Pangeo ({len(delta['member'])} models)")
plot.axes.coastlines()

plt.savefig("uncer-europe-pangeo.pdf", bbox_inches="tight")

Regional stripes.

In [ ]:
cmip6_ssp585_y = cmip6_ssp585_gwl["pr"].resample(time="QS-DEC").mean()
cmip6_ssp585_y = cmip6_ssp585_y.sel(time=cmip6_ssp585_y["time"].dt.month==12)

cmip6_hist_c = cmip6_historical_gwl["pr"].mean(["time"])

year_delta = cmip6_ssp585_y - cmip6_hist_c
year_delta_rel = year_delta / cmip6_hist_c * 100

regional_mean = year_delta_rel.mean(["lat", "lon"])

regional_mean.plot.imshow(
    figsize=(14,3.5),
    add_colorbar=True,
    cmap="BrBG",
    vmin=-50,vmax=50)
plt.savefig("regional-stripes-pr.pdf", bbox_inches="tight")

# Annex I

This functions optimizes dask chunking for efficient retrieval. However, it is even more expensive than loading unnecessary data (currently, historical takes ~13 min and with this function it takes ~23 min).

In [ ]:
def efficient_retrieval_and_compute(ds):
    lats, lons = slice(35, 72), slice(-11, 35) # Europe in (-180, 180)

    # Step 1: Map values to group labels
    grouped = ((ds["time"].dt.month.values >= 3) & (ds["time"].dt.month.values < 12)).astype(int)  # winter months 12, 1, 2
    change_indices = np.where(np.diff(grouped) != 0)[0] + 1 # Step 2: Find where the group changes
    segment_edges = np.concatenate(([0], change_indices, [len(ds["time"].dt.month.values)])) # Step 3: Segment edges
    counts = tuple(np.diff(segment_edges)) # Step 4: Lengths of each group run

    # handle Europe being split when using (0, 360)
    if ds["lon"].min().item() >= 0:
        left = ds.chunk({"time": counts}).sel(time=ds.time.dt.month.isin([12,1,2]), lon=slice(0, 36), lat=lats)
        right = ds.chunk({"time": counts}).sel(time=ds.time.dt.month.isin([12,1,2]), lon=slice(348, 360), lat=lats)

        spatial = xarray.concat([left, right], dim="lon")
    else:
        spatial = ds.sel(time=ds.time.dt.month.isin([12,1,2]), lon=lons, lat=lats)

    winter = spatial.resample({"time": "MS"}).sum().load()
    winter = winter.assign_coords(lon=((winter["lon"] + 180) % 360) - 180).sortby("lon")

    return winter